In [16]:
def parse_file(file_path, f_id):
    with open(file_path, "r") as file:
        lines = file.readlines()

    atoms = {}
    obstacles = {}
    win_condition_id = None
    interaction_pairs = {}
    level_mapping = {}
    rules = {}

    in_movable = in_obstacle = in_interaction_set = in_level_mapping = False

    for line in lines:
        line = line.strip()

        if "movable >" in line:
            in_movable = True
            continue
        elif "obstacle >" in line:
            in_obstacle = True
            in_movable = False
            continue
        elif line.startswith("InteractionSet"):
            in_interaction_set = True
            in_obstacle = False
        elif line.startswith("LevelMapping"):
            in_level_mapping = True
            in_interaction_set = False
        elif line.startswith("TerminationSet"):
            in_level_mapping = False

        if in_movable and "> Passive" in line:
            id = line.split()[0]
            atoms[id] = None
        elif in_obstacle and "> Immovable" in line:
            id = line.split()[0]
            obstacles[id] = None
        elif in_interaction_set and "> createSprite stype=" in line:
            parts = line.split(" > createSprite stype=")
            ids1, _ = parts
            id1, id2 = ids1.split(" ")
            left_side = (int(id1), int(id2))
            right_side = parts[1].strip('"')
            interaction_pairs[right_side] = left_side
            rules[left_side] = int(right_side)
        elif in_interaction_set and "> killBoth" in line:
            parts = line.split(" > killBoth")
            ids1, _ = parts
            id1, id2 = ids1.split(" ")
            left_side = (int(id1), int(id2))
            right_side = 99999
            rules[left_side] = right_side
        elif in_level_mapping and ">" in line:
            symbol, mapping = line.split(" > ")
            id = mapping.split()[-1]
            level_mapping[id] = symbol
        elif "MultiSpriteCounter stype=" in line:
            win_condition_id = line.split('"')[1]

    # Update atoms and obstacles with symbols from level mapping
    for id in atoms.keys():
        atoms[id] = level_mapping.get(id)
    for id in obstacles.keys():
        obstacles[id] = level_mapping.get(id)

    # Find the win condition based on the win_condition_id
    win_condition = interaction_pairs.get(win_condition_id, None)

    return {
        f_id: [
            {
                "walls": None,
                "atoms": atoms,
                "obstacles": obstacles,
                "agent": (None, None),
                "win_condition": {win_condition: int(win_condition_id)},
            },
            {"rules": rules},
        ]
    }


In [17]:
def update_dictionary_with_layout(file_path, dictionary, f_id):
    with open(file_path, "r") as file:
        layout = file.read()

    walls = []
    agent = None
    atoms = {}
    obstacles = {}

    layout_lines = layout.split("\n")
    for y, line in enumerate(layout_lines):
        for x, symbol in enumerate(line):
            if symbol == "w":
                walls.append((x, y))
            elif symbol == "A":
                agent = (x, y)
            else:
                for key, value in dictionary[f_id][0]["atoms"].items():
                    if value == symbol:
                        key = int(key)
                        atoms[(x, y)] = key
                for key, value in dictionary[f_id][0]["obstacles"].items():
                    if value == symbol:
                        key = int(key)
                        obstacles[(x, y)] = key
            size = (x+1, y+1)

    if atoms and obstacles:
        dictionary[f_id][0]["size"] = size
        dictionary[f_id][0]["walls"] = walls
        dictionary[f_id][0]["agent"] = agent
        dictionary[f_id][0]["atoms"] = atoms
        dictionary[f_id][0]["obstacles"] = obstacles

    return dictionary


In [137]:
def parse_files(def_file_path, layout_file_paths):
    dictionary = {}
    for i, layout_file_path in enumerate(layout_file_paths):
        if i not in dictionary:
            dictionary.update(parse_file(def_file_path, i))
        dictionary = update_dictionary_with_layout(layout_file_path, dictionary, i)
    return dictionary

In [145]:
import os

def_file_path = 'generated/generated_definition_2.txt'
layout_folder_path = 'layouts_2'

layout_file_paths = [os.path.join(layout_folder_path, f) for f in os.listdir(layout_folder_path) if os.path.isfile(os.path.join(layout_folder_path, f))]

dictionary = parse_files(def_file_path, layout_file_paths)


In [146]:
import pickle
with open("dic_2.pkl", "wb") as f:
    pickle.dump(dictionary, f)

In [147]:
def validate_agent_in_layout(dictionary):
    new_dic = {}
    for k, v in dictionary.items():
        if v[0]["agent"] is not None: 
            new_dic[k] = v 
    return new_dic


In [148]:
new_dic = validate_agent_in_layout(dictionary)

In [149]:
len(new_dic)

887

In [150]:
import pickle
with open("dic_2.pkl", "wb") as f:
    pickle.dump(new_dic, f)